In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from skimage import io
from scipy.stats import describe
from skimage.exposure import rescale_intensity, histogram, adjust_gamma
from flowdec import data as fd_data
import pims
# import pims
from skimage import io
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
from pims import ND2_Reader as nd2_two
import matplotlib.pyplot as plt


In [3]:
# channels = ['miRFP670', 'mRuby3','FITC']


In [4]:
fname = 'igfp1_caruby5_continue001trigger003.nd2'
# fname = '/home/jmamede/Area51/igfp3_caruby3_rutin_bvd_f2_2days_xy02.nd2'
# fname = '/tmp/test.ome.tiff'
# frames =  pims.ND2_Reader('/home/jmamede/Area51/igfp3_caruby3_rutin_bvd_f2_2days_xy02.nd2.')
frames =  frames = pims.bioformats.BioformatsReader(fname)
# fames2 = nd2_two(fname)
# frames = pims.bioformats.BioformatsReader(fname)
# frames.reader=0
#frames.iter_axes = 't'  # 't' is the default already
frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

/home/jmamede/anaconda3/lib/python3.7/site-packages/pims/bioformats.py:408: UserWarning: Due to an issue with JPype 0.6.0, reading is slower. Please consider upgrading JPype to 0.6.1 or later.
  warn('Due to an issue with JPype 0.6.0, reading is slower. '


In [7]:
frames.sizes
#['z']
#frames.bundle_axes = 'czyx'  # when 'z' is available, this will be default
#frames.iter_axes = 'c' 
#frames.default_coords['c'] = 1  # 0 is the default setting
#for frame in frames[:3]:
    # do something with 3D frames in channel 1
metadata =  frames.metadata
# print(frames.metadata.keys())
# print(frames.metadata['events'])
# print(fames2.metadata.keys())
# print(fames2.metadata.flush())
# print(frames.parser._

In [28]:
metadata.PlaneExposureTime(0,5)

'None'

In [103]:
from apeer_ometiff_library import omexmlClass
omexml = omexmlClass.OMEXML()
#example on how to add more series!
omexml.image_count = 8
p = omexml.image().Pixels
p7 = omexml.image(7).Pixels
# >>> o = OMEXML()
# >>> o.image().Pixels.channel_count = 3
# >>> o.image().Pixels.Channel(0).Name = "Red"
# >>> o.image().Pixels.Channel(1).Name = "Green"
# >>> o.image().Pixels.Channel(2).Name = "Blue"

In [81]:
def Nd2meta2OMEXML(nd2meta,output_file):
    from apeer_ometiff_library import omexmlClass
    
    def writeplanes(pixel, SizeT=1, SizeZ=1, SizeC=1, order='TZCYX', verbose=False):

        if order == 'TZCYX':

            pixel.DimensionOrder = omexmlClass.DO_XYCZT
#             counter = 0
            for t in range(SizeT):
                for z in range(SizeZ):
                    for c in range(SizeC):

                        if verbose:
                            print('Write PlaneTable: ', t, z, c),
                            sys.stdout.flush()

                        pixel.Plane(counter).TheT = t
                        pixel.Plane(counter).TheZ = z
                        pixel.Plane(counter).TheC = c
#                         counter = counter + 1

        return pixel
    SizeT = nd2meta['frames'][-1]+1
    SizeZ = nd2meta['z_levels'][-1]+1
    SizeC = len(nd2meta['channels'])
    SizeX = nd2meta['width']
    SizeY = nd2meta['height']
    print(SizeZ, SizeC, SizeX, SizeY)
    Series = nd2meta['fields_of_view'][-1]+1
    scalex = nd2meta['pixel_microns']
    scaley = scalex
    scalez = round(nd2meta['z_coordinates'][1]-nd2meta['z_coordinates'][0],3)
    pixeltype = 'uint16'
    dimorder = 'TZCYX'
# print(a)
    omexml = omexmlClass.OMEXML()
#     omexml.image(0).Name = output_file
    p = omexml.image(0).Pixels
    p.SizeX = SizeX
    p.SizeY = SizeY
    p.SizeC = SizeC
    p.SizeT = SizeT
    p.SizeZ = SizeZ
    p.PhysicalSizeX = np.float(scalex)
    p.PhysicalSizeY = np.float(scaley)
    p.PhysicalSizeZ = np.float(scalez)
    p.PixelType = pixeltype
    p.channel_count = SizeC
    p.plane_count = SizeZ * SizeT * SizeC #* SizeV
    p = writeplanes(p, SizeT=SizeT, SizeZ=SizeZ, SizeC=SizeC, order=dimorder)
    for c in range(SizeC):
        if pixeltype == 'unit8':
            p.Channel(c).SamplesPerPixel = 1
        if pixeltype == 'unit16':
            p.Channel(c).SamplesPerPixel = 2
        
    omexml.structured_annotations.add_original_metadata(omexmlClass.OM_SAMPLES_PER_PIXEL, str(SizeC))
    
    return omexml.to_xml()
    

In [82]:
beta = Nd2meta2OMEXML(nd2meta,'test.ome.tiff')
print(beta)

11 5 2048 2044
<ome:OME xmlns:om="openmicroscopy.org/OriginalMetadata" xmlns:ome="http://www.openmicroscopy.org/Schemas/OME/2016-06" xmlns:sa="http://www.openmicroscopy.org/Schemas/sa/2013-06s" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
    <ome:Image ID="Image:0" Name="default.png">
        <ome:AcquisitionDate>%(DEFAULT_NOW)s</ome:AcquisitionDate>
        <ome:Pixels BigEndian="false" DimensionOrder="XYCZT" ID="Pixels:0" Interleaved="false" PhysicalSizeX="0.108333333333333" PhysicalSizeY="0.108333333333333" PhysicalSizeZ="0.3" SizeC="5" SizeT="1" SizeX="2048" SizeY="2044" SizeZ="11" Type="uint16">
            <ome:Channel ID="Channel:0:0" SamplesPerPixel="1">
                <ome:LightPath />
            </ome:Channel>
        <ome:Channel ID="424ca862-d647-4e58-97a6-985d818dd26f" Name="424ca862-d647-4e58-97a6-985d818dd26f" SamplesPerPixel="1" /

In [83]:
SizeT = 1
SizeZ = 11
SizeC = 5
SizeX = 2048
SizeY = 2044
Series = 0



# create numpy array with correct order
img5d = np.random.randn(SizeT, SizeZ, SizeC, SizeY, SizeX).astype(np.uint16)

import tifffile as tf
tf.imsave('/tmp/test.ome.tiff',img5d, description = beta, metadata={'axes': 'TZCYX'})



In [13]:
# import datetime

import tifffile as tf
import tensorflow as tflow
# import json


for visit in range(frames.metadata.ImageCount()):
    #%timeit
    frames.series=visit
    with tf.TiffWriter('/home/jmamede/Area51/'+str(visit)+'.tif'
            img5d = np.swapaxes(img5d,0,1)
            tif.save(img5d
                    ,shape=img5d.shape

            #,resolution= (.1083,0.1083,3)
             , description = xml
            , photometric='minisblack'
            #, datetime= True
            , metadata={'axes': 'TZCYX', 'DimensionOrder' : 'TZCYX', 'Resolution': 0.1083}
                )
        tif.close()
#             res.append(algo.run(
#                 fd_data.Acquisition(
#                 data=frames[0],
#                 kernel=psf)
#                 , niter=20)
#                       )
            #reassumblethetiff.
            


Visit: 0
Time: 0


TypeError: 'MetadataRetrieve' object is not subscriptable

In [1]:
# tif.close()
from scipy.stats import describe
describe(frames[0].ravel())


NameError: name 'tif' is not defined

In [ ]:
plt.imshow(frames[0].max(axis=0),vmax=5000)

In [ ]:
res_stack = np.stack((res[1].data,np.zeros(res[1].data.shape), res[0].data))

In [ ]:
print(res_stack.shape)

In [ ]:
plt.imshow(res[1].data.max(axis=0),vmax=5000)

In [ ]:
plt.imshow(res[0].data[10],vmax=5000)

In [ ]:
from pims import Frame

zproj = res.data.max(axis=0)
frame = Frame(zproj
    )


#frame = Frame(res.data)
%matplotlib qt5

In [ ]:
plt.imshow(frame)

In [ ]:
plt.imshow(psf[0])

In [ ]:
print(meta)

In [ ]:
ctrl = pims.ND2_Reader('/home/jmamede/Area51/antibody4 - Deconvolved, Type Automatic.nd2')
ctrl.bundle_axes = 'zyx'
ctrl.default_coords['m'] = 0

print(ctrl[0].shape)
#frames.bundle_axes = 'czyx'
#frames.default_coords['m'] = 0

In [ ]:
from skimage import data
import napari
#res8 = (res_stack/256).astype('uint8')
with napari.gui_qt():
    viewer = napari.Viewer()
    viewer.add_image(res[1].data
                     , colormap = 'red'
                    )
    viewer.add_image(res[0].data
                     ,colormap = 'blue'
                    )
#     viewer.add_image(frames[0]
#                      , colormap = 'red'
#                     )
#     viewer.add_image(frames[2]
#                      ,colormap = 'blue'
#                     )
#     ctrl.default_coords['c'] = 1
#     viewer.add_image(ctrl[0]
#                      , colormap = 'red'
#                     )
#     ctrl.default_coords['c'] = 0
#     viewer.add_image(ctrl[0]
#                      ,colormap = 'blue'
#                     )

In [ ]:
res[1].data.shape

In [ ]:
ctrl[0].data.shape